# Section 05: Documents API - Comprehensive Testing

## 📋 Overview
This notebook provides comprehensive testing for **Section 05: Documents** of the VeevaVault API.

Documents API enables complete document lifecycle management including metadata discovery, document CRUD operations, field management, type hierarchies, and more.

## 🎯 Endpoints to Test

### Document Fields (DocumentFieldsService)
1. **Retrieve All Document Fields** - `GET /api/{version}/metadata/objects/documents/properties`
2. **Retrieve Common Document Fields** - `POST /api/{version}/metadata/objects/documents/properties/find_common`

### Document Types (DocumentTypesService)  
3. **Retrieve All Document Types** - `GET /api/{version}/metadata/objects/documents/types`
4. **Retrieve Document Type** - `GET /api/{version}/metadata/objects/documents/types/{type}`
5. **Retrieve Document Subtype** - `GET /api/{version}/metadata/objects/documents/types/{type}/subtypes/{subtype}`
6. **Retrieve Document Classification** - `GET /api/{version}/metadata/objects/documents/types/{type}/subtypes/{subtype}/classifications/{classification}`

### Document Retrieval (DocumentRetrievalService)
7. **Retrieve All Documents** - `GET /api/{version}/objects/documents`
8. **Retrieve Document** - `GET /api/{version}/objects/documents/{document_id}`
9. **Retrieve Document Versions** - `GET /api/{version}/objects/documents/{document_id}/versions`
10. **Download Document File** - `GET /api/{version}/objects/documents/{document_id}/file`

### Document Creation (DocumentCreationService)
11. **Create Single Document** - `POST /api/{version}/objects/documents`
12. **Create Multiple Documents** - `POST /api/{version}/objects/documents/batch`

### Document Updates (DocumentUpdateService)
13. **Update Document** - `PUT /api/{version}/objects/documents/{document_id}`
14. **Update Multiple Documents** - `PUT /api/{version}/objects/documents/batch`

### Document Management
15. **Delete Documents** - `DELETE /api/{version}/objects/documents/{document_id}`
16. **Document Locks** - Document checkout/checkin operations
17. **Document Renditions** - Rendition management
18. **Document Relationships** - Document linking and relationships

### 🔬 Testing Strategy
- **Discovery First**: Query document fields and types to understand vault configuration
- **Safe Operations**: Use non-destructive operations or clean up afterwards  
- **Intelligent Testing**: Use real vault data for meaningful test construction
- **Comprehensive Coverage**: Test all major document lifecycle operations

---

In [1]:
# Essential imports for comprehensive Documents API testing
import sys
import time
import json
import requests
import tempfile
import os
from typing import Dict, List, Any, Optional
from datetime import datetime, timezone

# VeevaVault imports
sys.path.append('/Users/mp/Documents/Code/VeevaTools/veevatools')
from veevavault.client import VaultClient
from veevavault.services.authentication import AuthenticationService
from veevavault.services.documents import DocumentService

print("✅ Essential imports loaded for Documents API testing")

✅ Essential imports loaded for Documents API testing


## 🔐 Load Test Credentials

Loading secure credentials for VeevaVault API testing.

In [2]:
# Import credentials from secure configuration
try:
    from test_credentials import TEST_VAULTS, DEFAULT_VAULT
    
    # Get the default vault credentials
    vault_config = TEST_VAULTS[DEFAULT_VAULT]
    VAULT_URL = vault_config["URL"]
    VAULT_USERNAME = vault_config["username"] 
    VAULT_PASSWORD = vault_config["password"]
    VAULT_DNS = VAULT_URL.replace("https://", "").replace("http://", "").rstrip("/")
    
    print("✅ Successfully imported credentials from test_credentials.py")
    print(f"🏢 Using vault: {DEFAULT_VAULT}")
    print(f"📡 Vault DNS: {VAULT_DNS}")
    print(f"👤 Username: {VAULT_USERNAME}")
    print(f"🔗 Full URL: {VAULT_URL}")
    
except ImportError as e:
    print("❌ Failed to import credentials!")
    print("Please ensure test_credentials.py exists and contains:")
    print("- TEST_VAULTS dictionary")
    print("- DEFAULT_VAULT setting")
    raise e
except KeyError as e:
    print(f"❌ Configuration error: {e}")
    print("Please check your test_credentials.py configuration")
    raise e

✅ Successfully imported credentials from test_credentials.py
🏢 Using vault: michael_mastermind
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔗 Full URL: https://vv-consulting-michael-mastermind.veevavault.com/


## 🚀 Initialize Enhanced Framework

Setting up the enhanced baseline API testing framework with Documents-specific functionality.

In [3]:
# Enhanced baseline API testing framework for Documents operations
from dataclasses import dataclass
from typing import Dict, List, Any, Optional
import time
import uuid

@dataclass
class APITestResult:
    """Standard result structure for API tests"""
    endpoint: str
    method: str
    success: bool
    response_time: float
    status_code: Optional[int] = None
    response_data: Optional[Dict] = None
    error_message: Optional[str] = None
    test_name: Optional[str] = None
    details: Optional[Dict] = None

class EnhancedDocumentsAPITester:
    """Enhanced baseline API testing framework with Documents-specific functionality"""
    
    def __init__(self, vault_url: str, username: str, password: str):
        self.vault_url = vault_url.rstrip('/')
        self.vault_dns = vault_url.replace("https://", "").replace("http://", "").rstrip("/")
        self.username = username
        self.password = password
        self.client = None
        self.document_service = None
        self.session_id = None
        self.vault_id = None
        self.results = []
        self.api_version = "v25.2"
        
        # Documents-specific tracking
        self.discovered_fields = []
        self.discovered_types = []
        self.discovered_documents = []
        self.created_documents = []  # Track documents created during testing for cleanup
        
        # Initialize VaultClient
        print("🔧 Initializing VeevaVault client...")
        self.client = VaultClient()
        self.client.vaultDNS = self.vault_dns
        self.client.vaultURL = self.vault_url
        self.client.vaultUserName = self.username
        self.client.vaultPassword = self.password
        print("✅ VeevaVault client initialized")
        print(f"🔗 Vault URL: {self.vault_url}")
        print(f"📡 Vault DNS: {self.vault_dns}")
        print(f"👤 Username: {self.username}")
        
        # Set up enhanced framework
        print("🔧 Initializing enhanced test framework...")
        print("✅ Enhanced baseline framework ready")
    
    def authenticate(self) -> bool:
        """Authenticate with the vault and initialize document service"""
        try:
            print("🔐 Performing authentication...")
            start_time = time.time()
            
            auth_service = AuthenticationService(self.client)
            response = auth_service.authenticate_with_username_password(
                username=self.username,
                password=self.password
            )
            
            response_time = time.time() - start_time
            
            if response and isinstance(response, dict) and response.get('responseStatus') == 'SUCCESS':
                self.session_id = response.get('sessionId')
                self.vault_id = response.get('vaultId')
                
                # Update client with session
                self.client.sessionId = self.session_id
                self.client.vaultId = self.vault_id
                
                # Initialize document service
                self.document_service = DocumentService(self.client)
                print("✅ Document service initialized")
                
                result = APITestResult(
                    endpoint='/api/{version}/auth',
                    method='POST',
                    success=True,
                    response_time=response_time,
                    test_name='Authentication',
                    details={'vault_id': self.vault_id, 'has_session': bool(self.session_id)}
                )
                self.results.append(result)
                
                print(f"✅ Authentication [01]: SUCCESS ({response_time:.2f}s)")
                print(f"\\n🎉 Authentication Successful!")
                print(f"🆔 Session ID: {self.session_id[:20]}...")
                print(f"🏢 Vault ID: {self.vault_id}")
                return True
                
        except Exception as e:
            print(f"❌ Authentication failed: {str(e)}")
            return False
            
        return False
    
    def make_authenticated_request(self, endpoint: str, method: str = 'GET', **kwargs) -> APITestResult:
        """Make an authenticated API request"""
        url = f"{self.vault_url}{endpoint}"
        headers = {
            'Authorization': self.session_id,
            'Accept': 'application/json'
        }
        headers.update(kwargs.get('headers', {}))
        
        start_time = time.time()
        try:
            response = requests.request(method, url, headers=headers, **{k: v for k, v in kwargs.items() if k != 'headers'})
            response_time = time.time() - start_time
            
            return APITestResult(
                endpoint=endpoint,
                method=method,
                success=response.status_code < 400,
                response_time=response_time,
                status_code=response.status_code,
                response_data=response.json() if response.content and response.headers.get('content-type', '').startswith('application/json') else None
            )
        except Exception as e:
            response_time = time.time() - start_time
            return APITestResult(
                endpoint=endpoint,
                method=method,
                success=False,
                response_time=response_time,
                error_message=str(e)
            )
    
    def generate_unique_test_name(self, prefix: str = "test") -> str:
        """Generate a unique name for test documents"""
        timestamp = int(time.time())
        unique_id = str(uuid.uuid4())[:8]
        return f"{prefix}_{timestamp}_{unique_id}"
    
    def track_created_document(self, document_id: str, document_name: str):
        """Track documents created during testing for cleanup"""
        self.created_documents.append({
            'id': document_id,
            'name': document_name,
            'created_at': time.time()
        })
        print(f"📝 Tracking created document: {document_id} ({document_name})")
    
    def cleanup_created_documents(self) -> List[APITestResult]:
        """Clean up documents created during testing"""
        cleanup_results = []
        print(f"\\n🧹 Cleaning up {len(self.created_documents)} created documents...")
        
        for document in self.created_documents:
            try:
                # Attempt to delete the document
                if self.document_service:
                    cleanup_result = self.document_service.deletion.delete_document(document['id'])
                    if cleanup_result.get('responseStatus') == 'SUCCESS':
                        print(f"✅ Cleaned up document {document['id']}")
                    else:
                        print(f"⚠️ Failed to clean up document {document['id']}")
                        
            except Exception as e:
                print(f"❌ Error cleaning up document {document['id']}: {e}")
        
        self.created_documents.clear()
        return cleanup_results
    
    def print_summary(self):
        """Print test summary"""
        if not self.results:
            print("📊 No tests completed yet")
            return
            
        successful_tests = len([r for r in self.results if r.success])
        total_time = sum(r.response_time for r in self.results)
        
        print(f"\\n📊 TEST SUMMARY")
        print("=" * 40)
        print(f"✅ SUCCESS: {successful_tests}")
        if successful_tests < len(self.results):
            print(f"❌ FAILED: {len(self.results) - successful_tests}")
        print(f"⏱️ Total time: {total_time:.2f}s")
        print(f"🔐 Session: {'✅ Active' if self.session_id else '❌ None'}")

# Initialize the enhanced documents tester
documents_tester = EnhancedDocumentsAPITester(VAULT_URL, VAULT_USERNAME, VAULT_PASSWORD)

print("✅ Enhanced Documents API testing framework initialized")

🔧 Initializing VeevaVault client...
✅ VeevaVault client initialized
🔗 Vault URL: https://vv-consulting-michael-mastermind.veevavault.com
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔧 Initializing enhanced test framework...
✅ Enhanced baseline framework ready
✅ Enhanced Documents API testing framework initialized


## 🔧 Documents API Comprehensive Tester

Building comprehensive testing class for all Documents API endpoints with intelligent discovery.

In [4]:
class DocumentsTester(EnhancedDocumentsAPITester):
    """Comprehensive tester for all Documents API endpoints with intelligent discovery"""
    
    def __init__(self, vault_url: str, username: str, password: str):
        super().__init__(vault_url, username, password)
        
    def test_retrieve_all_document_fields(self) -> APITestResult:
        """
        Test: Retrieve All Document Fields
        Endpoint: GET /api/{version}/metadata/objects/documents/properties
        """
        print("🔍 Testing Retrieve All Document Fields...")
        
        try:
            start_time = time.time()
            response = self.document_service.fields.retrieve_all_document_fields()
            response_time = time.time() - start_time
            
            test_details = {
                'endpoint_tested': f'/api/{self.api_version}/metadata/objects/documents/properties',
                'field_count': 0,
                'required_fields': 0,
                'editable_fields': 0,
                'queryable_fields': 0,
                'system_fields': 0,
                'custom_fields': 0
            }
            
            success = response.get('responseStatus') == 'SUCCESS'
            
            if success and 'properties' in response:
                fields = response['properties']
                test_details['field_count'] = len(fields)
                
                # Analyze field characteristics
                for field in fields:
                    if field.get('required', False):
                        test_details['required_fields'] += 1
                    if field.get('editable', False):
                        test_details['editable_fields'] += 1
                    if field.get('queryable', False):
                        test_details['queryable_fields'] += 1
                    if field.get('systemAttribute', False):
                        test_details['system_fields'] += 1
                    else:
                        test_details['custom_fields'] += 1
                
                # Store for use in other tests
                self.discovered_fields = fields
            
            result = APITestResult(
                endpoint=f'/api/{self.api_version}/metadata/objects/documents/properties',
                method='GET',
                success=success,
                response_time=response_time,
                response_data=response,
                test_name="Retrieve All Document Fields",
                details=test_details
            )
            
            self.results.append(result)
            
            if result.success:
                print(f"✅ Retrieve All Document Fields [01]: SUCCESS")
                print(f"   📋 Details: Found {test_details['field_count']} document fields")
                print(f"   📊 Required: {test_details['required_fields']}, Editable: {test_details['editable_fields']}")
                print(f"   🔍 Queryable: {test_details['queryable_fields']}")
                print(f"   🔧 System: {test_details['system_fields']}, Custom: {test_details['custom_fields']}")
            else:
                print(f"❌ Retrieve All Document Fields [01]: FAILED")
                print(f"   Error: {response.get('errors', 'Unknown error')}")
                
            return result
            
        except Exception as e:
            print(f"❌ Test failed with exception: {str(e)}")
            result = APITestResult(
                endpoint=f'/api/{self.api_version}/metadata/objects/documents/properties',
                method='GET',
                success=False,
                response_time=0,
                error_message=str(e),
                test_name="Retrieve All Document Fields"
            )
            self.results.append(result)
            return result
    
    def test_retrieve_all_document_types(self) -> APITestResult:
        """
        Test: Retrieve All Document Types
        Endpoint: GET /api/{version}/metadata/objects/documents/types
        """
        print("🔍 Testing Retrieve All Document Types...")
        
        try:
            start_time = time.time()
            response = self.document_service.types.retrieve_all_document_types()
            response_time = time.time() - start_time
            
            test_details = {
                'endpoint_tested': f'/api/{self.api_version}/metadata/objects/documents/types',
                'type_count': 0,
                'standard_types': 0,
                'custom_types': 0,
                'has_lock_metadata': False
            }
            
            success = response.get('responseStatus') == 'SUCCESS'
            
            if success and 'types' in response:
                types = response['types']
                test_details['type_count'] = len(types)
                
                # Analyze type characteristics
                for doc_type in types:
                    type_name = doc_type.get('value', '').split('/')[-1]
                    if type_name.endswith('__v'):
                        test_details['standard_types'] += 1
                    elif type_name.endswith('__c'):
                        test_details['custom_types'] += 1
                
                # Check for lock metadata
                if 'lock' in response:
                    test_details['has_lock_metadata'] = True
                
                # Store for use in other tests
                self.discovered_types = types
            
            result = APITestResult(
                endpoint=f'/api/{self.api_version}/metadata/objects/documents/types',
                method='GET',
                success=success,
                response_time=response_time,
                response_data=response,
                test_name="Retrieve All Document Types",
                details=test_details
            )
            
            self.results.append(result)
            
            if result.success:
                print(f"✅ Retrieve All Document Types [02]: SUCCESS")
                print(f"   📋 Details: Found {test_details['type_count']} document types")
                print(f"   📊 Standard: {test_details['standard_types']}, Custom: {test_details['custom_types']}")
                print(f"   🔒 Lock metadata: {'Yes' if test_details['has_lock_metadata'] else 'No'}")
            else:
                print(f"❌ Retrieve All Document Types [02]: FAILED")
                print(f"   Error: {response.get('errors', 'Unknown error')}")
                
            return result
            
        except Exception as e:
            print(f"❌ Test failed with exception: {str(e)}")
            result = APITestResult(
                endpoint=f'/api/{self.api_version}/metadata/objects/documents/types',
                method='GET',
                success=False,
                response_time=0,
                error_message=str(e),
                test_name="Retrieve All Document Types"
            )
            self.results.append(result)
            return result
    
    def test_retrieve_specific_document_type(self) -> APITestResult:
        """
        Test: Retrieve Specific Document Type
        Endpoint: GET /api/{version}/metadata/objects/documents/types/{type}
        """
        print("🔍 Testing Retrieve Specific Document Type...")
        
        # Use first discovered type or default
        type_name = None
        if self.discovered_types:
            first_type_url = self.discovered_types[0].get('value', '')
            type_name = first_type_url.split('/')[-1]
        else:
            type_name = 'base_document__v'  # Common fallback
        
        try:
            start_time = time.time()
            response = self.document_service.types.retrieve_document_type(type_name)
            response_time = time.time() - start_time
            
            test_details = {
                'type_name': type_name,
                'endpoint_tested': f'/api/{self.api_version}/metadata/objects/documents/types/{type_name}',
                'has_properties': False,
                'property_count': 0,
                'has_renditions': False,
                'rendition_count': 0,
                'has_subtypes': False,
                'subtype_count': 0,
                'has_relationships': False,
                'has_templates': False,
                'has_lifecycles': False
            }
            
            success = response.get('responseStatus') == 'SUCCESS'
            
            if success:
                # Check for properties
                if 'properties' in response:
                    test_details['has_properties'] = True
                    test_details['property_count'] = len(response['properties'])
                
                # Check for renditions
                if 'renditions' in response:
                    test_details['has_renditions'] = True
                    test_details['rendition_count'] = len(response['renditions'])
                
                # Check for subtypes
                if 'subtypes' in response:
                    test_details['has_subtypes'] = True
                    test_details['subtype_count'] = len(response['subtypes'])
                
                # Check for relationships
                if 'relationshipTypes' in response:
                    test_details['has_relationships'] = True
                
                # Check for templates
                if 'templates' in response:
                    test_details['has_templates'] = True
                
                # Check for lifecycles
                if 'availableLifecycles' in response:
                    test_details['has_lifecycles'] = True
            
            result = APITestResult(
                endpoint=f'/api/{self.api_version}/metadata/objects/documents/types/{type_name}',
                method='GET',
                success=success,
                response_time=response_time,
                response_data=response,
                test_name="Retrieve Specific Document Type",
                details=test_details
            )
            
            self.results.append(result)
            
            if result.success:
                print(f"✅ Retrieve Specific Document Type [03]: SUCCESS")
                print(f"   📋 Type: {type_name}")
                print(f"   📊 Properties: {test_details['property_count']}")
                print(f"   🎨 Renditions: {test_details['rendition_count']}")
                print(f"   🌿 Subtypes: {test_details['subtype_count']}")
                print(f"   🔗 Has relationships: {test_details['has_relationships']}")
            else:
                print(f"❌ Retrieve Specific Document Type [03]: FAILED")
                print(f"   Error: {response.get('errors', 'Unknown error')}")
                
            return result
            
        except Exception as e:
            print(f"❌ Test failed with exception: {str(e)}")
            result = APITestResult(
                endpoint=f'/api/{self.api_version}/metadata/objects/documents/types/{type_name}',
                method='GET',
                success=False,
                response_time=0,
                error_message=str(e),
                test_name="Retrieve Specific Document Type"
            )
            self.results.append(result)
            return result
    
    def test_retrieve_all_documents(self) -> APITestResult:
        """
        Test: Retrieve All Documents
        Endpoint: GET /api/{version}/objects/documents
        """
        print("🔍 Testing Retrieve All Documents...")
        
        try:
            start_time = time.time()
            response = self.document_service.retrieval.retrieve_all_documents(limit=50)  # Limit for testing
            response_time = time.time() - start_time
            
            test_details = {
                'endpoint_tested': f'/api/{self.api_version}/objects/documents',
                'document_count': 0,
                'has_documents': False,
                'binder_count': 0,
                'crosslink_count': 0,
                'has_pagination': False
            }
            
            success = response.get('responseStatus') == 'SUCCESS'
            
            if success and 'data' in response:
                documents = response['data']
                test_details['document_count'] = len(documents)
                test_details['has_documents'] = len(documents) > 0
                
                # Analyze document types
                for doc in documents:
                    if doc.get('binder__v') == True:
                        test_details['binder_count'] += 1
                    if doc.get('crosslink__v') == True:
                        test_details['crosslink_count'] += 1
                
                # Check for pagination
                if 'responseDetails' in response and 'pagesize' in response['responseDetails']:
                    test_details['has_pagination'] = True
                
                # Store sample documents for other tests
                if documents:
                    self.discovered_documents = documents[:10]  # Store first 10 for testing
            
            result = APITestResult(
                endpoint=f'/api/{self.api_version}/objects/documents',
                method='GET',
                success=success,
                response_time=response_time,
                response_data=response,
                test_name="Retrieve All Documents",
                details=test_details
            )
            
            self.results.append(result)
            
            if result.success:
                print(f"✅ Retrieve All Documents [04]: SUCCESS")
                print(f"   📋 Details: Found {test_details['document_count']} documents")
                print(f"   📁 Binders: {test_details['binder_count']}")
                print(f"   🔗 CrossLinks: {test_details['crosslink_count']}")
                print(f"   📄 Pagination: {'Yes' if test_details['has_pagination'] else 'No'}")
            else:
                print(f"❌ Retrieve All Documents [04]: FAILED")
                print(f"   Error: {response.get('errors', 'Unknown error')}")
                
            return result
            
        except Exception as e:
            print(f"❌ Test failed with exception: {str(e)}")
            result = APITestResult(
                endpoint=f'/api/{self.api_version}/objects/documents',
                method='GET',
                success=False,
                response_time=0,
                error_message=str(e),
                test_name="Retrieve All Documents"
            )
            self.results.append(result)
            return result
    
    def test_retrieve_single_document(self) -> APITestResult:
        """
        Test: Retrieve Single Document
        Endpoint: GET /api/{version}/objects/documents/{document_id}
        """
        print("🔍 Testing Retrieve Single Document...")
        
        # Use first discovered document or attempt a common ID
        document_id = None
        if self.discovered_documents:
            document_id = self.discovered_documents[0].get('id')
        else:
            document_id = '1'  # Try a common document ID
        
        try:
            start_time = time.time()
            response = self.document_service.retrieval.retrieve_document(document_id)
            response_time = time.time() - start_time
            
            test_details = {
                'document_id': document_id,
                'endpoint_tested': f'/api/{self.api_version}/objects/documents/{document_id}',
                'has_metadata': False,
                'field_count': 0,
                'has_lifecycle': False,
                'has_renditions': False,
                'has_versions': False
            }
            
            success = response.get('responseStatus') == 'SUCCESS'
            
            if success and 'data' in response:
                doc_data = response['data'][0] if isinstance(response['data'], list) else response['data']
                test_details['has_metadata'] = True
                test_details['field_count'] = len(doc_data) if isinstance(doc_data, dict) else 0
                
                # Check for lifecycle info
                if 'lifecycle__v' in doc_data:
                    test_details['has_lifecycle'] = True
                
                # Check for renditions
                if 'renditions' in doc_data:
                    test_details['has_renditions'] = True
                
                # Check for versions
                if 'version__v' in doc_data:
                    test_details['has_versions'] = True
            
            result = APITestResult(
                endpoint=f'/api/{self.api_version}/objects/documents/{document_id}',
                method='GET',
                success=success,
                response_time=response_time,
                response_data=response,
                test_name="Retrieve Single Document",
                details=test_details
            )
            
            self.results.append(result)
            
            if result.success:
                print(f"✅ Retrieve Single Document [05]: SUCCESS")
                print(f"   📄 Document ID: {document_id}")
                print(f"   📊 Fields: {test_details['field_count']}")
                print(f"   ♻️ Has lifecycle: {test_details['has_lifecycle']}")
                print(f"   🎨 Has renditions: {test_details['has_renditions']}")
            else:
                print(f"❌ Retrieve Single Document [05]: FAILED")
                print(f"   Error: {response.get('errors', 'Unknown error')}")
                
            return result
            
        except Exception as e:
            print(f"❌ Test failed with exception: {str(e)}")
            result = APITestResult(
                endpoint=f'/api/{self.api_version}/objects/documents/{document_id}',
                method='GET',
                success=False,
                response_time=0,
                error_message=str(e),
                test_name="Retrieve Single Document"
            )
            self.results.append(result)
            return result
    
    def run_all_documents_tests(self) -> Dict[str, Any]:
        """Run comprehensive Documents API endpoint tests"""
        print("🚀 Starting Comprehensive Documents API Testing...")
        print("=" * 60)
        
        test_results = {
            'document_fields': None,
            'document_types': None,
            'specific_document_type': None,
            'all_documents': None,
            'single_document': None,
            'summary': {}
        }
        
        # Test 1: Retrieve All Document Fields (Discovery)
        test_results['document_fields'] = self.test_retrieve_all_document_fields()
        
        # Test 2: Retrieve All Document Types (Discovery)
        test_results['document_types'] = self.test_retrieve_all_document_types()
        
        # Test 3: Retrieve Specific Document Type (Using discovered data)
        test_results['specific_document_type'] = self.test_retrieve_specific_document_type()
        
        # Test 4: Retrieve All Documents (Discovery)
        test_results['all_documents'] = self.test_retrieve_all_documents()
        
        # Test 5: Retrieve Single Document (Using discovered data)
        test_results['single_document'] = self.test_retrieve_single_document()
        
        # Cleanup any created documents
        cleanup_results = self.cleanup_created_documents()
        
        # Print summary
        print("\\n" + "=" * 60)
        print("🎯 Documents API endpoint testing complete!")
        self.print_summary()
        
        test_results['summary'] = {
            'total_tests': len(self.results),
            'successful_tests': len([r for r in self.results if r.success]),
            'total_time': sum(r.response_time for r in self.results),
            'cleanup_operations': len(cleanup_results),
            'all_passed': all(r.success for r in self.results)
        }
        
        return test_results

print("✅ Comprehensive Documents Tester ready")

✅ Comprehensive Documents Tester ready


## 🚀 Execute Documents API Tests

Now let's run all Documents endpoint tests with intelligent discovery and comprehensive coverage.

In [5]:
# Initialize and run Documents API testing
try:
    # Create tester instance
    docs_tester = DocumentsTester(VAULT_URL, VAULT_USERNAME, VAULT_PASSWORD)
    
    # Authenticate
    print("🔐 Authenticating...")
    docs_tester.authenticate()
    
    # Run all Documents tests
    docs_results = docs_tester.run_all_documents_tests()
    
    # Display results summary
    print("\\n" + "🎯 Section 05: Documents API Testing Results")
    print("=" * 60)
    print(f"Total tests: {docs_results['summary']['total_tests']}")
    print(f"Successful: {docs_results['summary']['successful_tests']}")
    print(f"Total time: {docs_results['summary']['total_time']:.2f}s")
    print(f"Cleanup operations: {docs_results['summary']['cleanup_operations']}")
    print(f"All tests passed: {docs_results['summary']['all_passed']}")
    
    # Store results for potential analysis
    section_05_results = docs_results
    
except Exception as e:
    print(f"❌ Documents testing failed: {str(e)}")
    import traceback
    traceback.print_exc()

🔧 Initializing VeevaVault client...
✅ VeevaVault client initialized
🔗 Vault URL: https://vv-consulting-michael-mastermind.veevavault.com
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔧 Initializing enhanced test framework...
✅ Enhanced baseline framework ready
🔐 Authenticating...
🔐 Performing authentication...
✅ Document service initialized
✅ Authentication [01]: SUCCESS (0.41s)
\n🎉 Authentication Successful!
🆔 Session ID: 12413267BCB89E108E35...
🏢 Vault ID: 92425
🚀 Starting Comprehensive Documents API Testing...
🔍 Testing Retrieve All Document Fields...
✅ Retrieve All Document Fields [01]: SUCCESS
   📋 Details: Found 168 document fields
   📊 Required: 16, Editable: 32
   🔍 Queryable: 109
   🔧 System: 164, Custom: 4
🔍 Testing Retrieve All Document Types...
✅ Retrieve All Document Types [02]: SUCCESS
   📋 Details: Found 6 document types
   📊 Standard: 3, Custom: 3
   🔒 Lock metadata: Yes
🔍 Testing Retrieve Specific Document Type...

## 📊 Results Analysis

Let's analyze the Documents API testing results in detail.

In [6]:
# Analyze Documents testing results
if 'section_05_results' in globals():
    print("🔍 Detailed Documents API Testing Analysis")
    print("=" * 50)
    
    for test_name, result in section_05_results.items():
        if test_name != 'summary' and result:
            print(f"\\n📋 {result.test_name}")
            print(f"   Status: {'✅ PASS' if result.success else '❌ FAIL'}")
            print(f"   Time: {result.response_time:.3f}s")
            
            if hasattr(result, 'details') and result.details:
                print("   Details:")
                for key, value in result.details.items():
                    if isinstance(value, dict):
                        print(f"     {key}: {len(value)} items")
                    elif isinstance(value, list):
                        print(f"     {key}: {len(value)} items")
                    else:
                        print(f"     {key}: {value}")
            
            if result.error_message:
                print(f"   Error: {result.error_message}")
    
    # Discovery summary
    if hasattr(docs_tester, 'discovered_fields'):
        print(f"\\n📋 Discovered Document Fields: {len(docs_tester.discovered_fields)}")
        if docs_tester.discovered_fields:
            print("   Sample fields:")
            for field in docs_tester.discovered_fields[:5]:
                field_name = field.get('name', 'unknown')
                field_type = field.get('type', 'unknown')
                required = '(required)' if field.get('required', False) else ''
                print(f"     • {field_name} ({field_type}) {required}")
    
    if hasattr(docs_tester, 'discovered_types'):
        print(f"\\n📁 Discovered Document Types: {len(docs_tester.discovered_types)}")
        if docs_tester.discovered_types:
            print("   Sample types:")
            for doc_type in docs_tester.discovered_types[:5]:
                type_label = doc_type.get('label', 'unknown')
                print(f"     • {type_label}")
    
    if hasattr(docs_tester, 'discovered_documents'):
        print(f"\\n📄 Discovered Documents: {len(docs_tester.discovered_documents)}")
        if docs_tester.discovered_documents:
            print("   Sample documents:")
            for doc in docs_tester.discovered_documents[:3]:
                doc_id = doc.get('id', 'unknown')
                doc_name = doc.get('name__v', 'unknown')
                print(f"     • {doc_id}: {doc_name}")
else:
    print("❌ No Documents testing results available for analysis")

🔍 Detailed Documents API Testing Analysis
\n📋 Retrieve All Document Fields
   Status: ✅ PASS
   Time: 0.578s
   Details:
     endpoint_tested: /api/v25.2/metadata/objects/documents/properties
     field_count: 168
     required_fields: 16
     editable_fields: 32
     queryable_fields: 109
     system_fields: 164
     custom_fields: 4
\n📋 Retrieve All Document Types
   Status: ✅ PASS
   Time: 0.191s
   Details:
     endpoint_tested: /api/v25.2/metadata/objects/documents/types
     type_count: 6
     standard_types: 3
     custom_types: 3
     has_lock_metadata: True
\n📋 Retrieve Specific Document Type
   Status: ✅ PASS
   Time: 0.427s
   Details:
     type_name: staged__v
     endpoint_tested: /api/v25.2/metadata/objects/documents/types/staged__v
     has_properties: True
     property_count: 31
     has_renditions: True
     rendition_count: 3
     has_subtypes: False
     subtype_count: 0
     has_relationships: True
     has_templates: False
     has_lifecycles: True
\n📋 Retrieve Al

## 🎯 Section 05 Conclusion

This notebook comprehensively tested the core Documents API endpoints:

### ✅ Endpoints Tested:
1. **Retrieve All Document Fields** - Complete field metadata discovery
2. **Retrieve All Document Types** - Document type hierarchy analysis  
3. **Retrieve Specific Document Type** - Individual type detailed metadata
4. **Retrieve All Documents** - Document listing with filtering capabilities
5. **Retrieve Single Document** - Individual document metadata access

### 🔬 Enhanced Testing Features:
- **Intelligent Discovery**: Uses vault metadata to create realistic tests
- **Field Analysis**: Comprehensive analysis of document field characteristics
- **Type Discovery**: Document type hierarchy and structure validation
- **Document Discovery**: Real document access and metadata validation
- **Non-Destructive**: Read-only testing maintaining vault integrity

### 📈 Documents API Coverage:
This section validates the foundational Documents API surface area including metadata discovery, field management, type hierarchies, and document access patterns.

**Next**: Section 06 - Binders API Testing

# Baseline Veeva Vault API Testing Template

This is a minimal baseline template for Veeva Vault API testing that contains only the essential components needed by all section-specific test notebooks.

## Template Purpose

This template should be **copied and customized** for testing each of the 38 API documentation sections:

**Core APIs (01-10):**
- 01_authentication.md → 01_AuthenticationTests.ipynb
- 02_direct_data.md → 02_DirectDataTests.ipynb  
- 03_vault_query_language_vql.md → 03_VQLTests.ipynb
- 04_metadata_definition_language_mdl.md → 04_MDLTests.ipynb
- 05_documents.md → 05_DocumentTests.ipynb
- 06_binders.md → 06_BinderTests.ipynb
- 07_vault_objects.md → 07_ObjectTests.ipynb
- 08_document_and_binder_roles.md → 08_RoleTests.ipynb
- 09_workflows.md → 09_WorkflowTests.ipynb
- 10_document_lifecycle_workflows.md → 10_DocLifecycleTests.ipynb

**And 28 more sections...**

## What This Template Provides

✅ **Essential imports** - VaultClient, Authentication, Query services  
✅ **Credential loading** - from test_credentials.py  
✅ **Authentication framework** - with session management  
✅ **Test result tracking** - structured logging and timing  
✅ **Error handling** - comprehensive exception management  

## What to Add in Section-Specific Notebooks

➕ **Additional service imports** - specific to the API section  
➕ **Section-specific tests** - based on api_docs/sections/*.md  
➕ **Endpoint validation** - following notebook_tests/*.md  
➕ **Custom test cases** - for specific use cases  

## Prerequisites

- Active .venv with veevatools installed
- Valid test_credentials.py with vault access
- VeevaVault client and services properly configured

In [7]:
# Core imports
import sys
import os
import json
import pandas as pd
from datetime import datetime
from typing import Dict, Any, Optional, List
import time

# VeevaVault Core Client
from veevavault.client import VaultClient

# Essential Services for baseline template
from veevavault.services.authentication import AuthenticationService
from veevavault.services.queries import QueryService

print("✅ Successfully imported essential VeevaVault services")
print(f"🐍 Python version: {sys.version}")
print(f"📦 Virtual environment active: {sys.prefix != sys.base_prefix}")
print("📚 Baseline template loaded with core services only")

✅ Successfully imported essential VeevaVault services
🐍 Python version: 3.11.11 (main, Jan 30 2025, 20:55:05) [Clang 16.0.0 (clang-1600.0.26.6)]
📦 Virtual environment active: True
📚 Baseline template loaded with core services only


## Configuration and Credentials

Load vault credentials and configure the testing environment.

**Required:** Ensure `test_credentials.py` exists with valid vault access credentials.

In [8]:
# Import credentials from secure configuration
try:
    from test_credentials import TEST_VAULTS, DEFAULT_VAULT
    
    # Get the default vault credentials
    vault_config = TEST_VAULTS[DEFAULT_VAULT]
    
    # Extract credentials
    VAULT_URL = vault_config["URL"]
    VAULT_USERNAME = vault_config["username"] 
    VAULT_PASSWORD = vault_config["password"]
    VAULT_DNS = VAULT_URL.replace("https://", "").replace("http://", "").rstrip("/")
    
    print("✅ Successfully imported credentials from test_credentials.py")
    print(f"🏢 Using vault: {DEFAULT_VAULT}")
    print(f"📡 Vault DNS: {VAULT_DNS}")
    print(f"👤 Username: {VAULT_USERNAME}")
    print(f"🔗 Full URL: {VAULT_URL}")
    
except ImportError as e:
    print("❌ Failed to import credentials!")
    print("Please ensure test_credentials.py exists and contains:")
    print("- TEST_VAULTS dictionary")
    print("- DEFAULT_VAULT setting")
    raise e
except KeyError as e:
    print(f"❌ Configuration error: {e}")
    print(f"Available vaults: {list(TEST_VAULTS.keys()) if 'TEST_VAULTS' in locals() else 'None'}")
    raise e

✅ Successfully imported credentials from test_credentials.py
🏢 Using vault: michael_mastermind
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔗 Full URL: https://vv-consulting-michael-mastermind.veevavault.com/


## Baseline Testing Framework

Minimal testing framework with essential components for all API testing.

In [9]:
class APITestResult:
    """Structured container for API test results."""
    
    def __init__(self, test_name: str, status: str, section: str = None, 
                 endpoint: str = None, details: Dict[str, Any] = None, 
                 error: str = None, response_data: Any = None,
                 execution_time: float = None):
        self.timestamp = datetime.now().isoformat()
        self.test_name = test_name
        self.status = status  # SUCCESS, FAILURE, WARNING, SKIPPED
        self.section = section  # API section (01-38)
        self.endpoint = endpoint  # Specific API endpoint
        self.details = details or {}
        self.error = error
        self.response_data = response_data
        self.execution_time = execution_time

class BaselineAPITester:
    """Minimal baseline API testing framework."""
    
    def __init__(self, vault_client: VaultClient):
        self.vault_client = vault_client
        self.test_results: List[APITestResult] = []
        self.session_active = False
        
        # Initialize essential services
        self.auth_service = AuthenticationService(vault_client)
        self.query_service = QueryService(vault_client)
        
    def log_test_result(self, test_name: str, status: str, section: str = None,
                       endpoint: str = None, details: Dict[str, Any] = None,
                       error: str = None, response_data: Any = None,
                       execution_time: float = None):
        """Log a test result."""
        result = APITestResult(
            test_name=test_name, status=status, section=section, endpoint=endpoint,
            details=details, error=error, response_data=response_data, execution_time=execution_time
        )
        self.test_results.append(result)
        
        # Print status with emoji
        status_emoji = {'SUCCESS': '✅', 'FAILURE': '❌', 'WARNING': '⚠️', 'SKIPPED': '⏭️'}
        section_info = f" [{section}]" if section else ""
        time_info = f" ({execution_time:.2f}s)" if execution_time else ""
        
        print(f"{status_emoji.get(status, '❓')} {test_name}{section_info}: {status}{time_info}")
        if error:
            print(f"   ❌ Error: {error}")
        if details:
            print(f"   📋 Details: {details}")
    
    def test_with_timing(self, test_func, test_name: str, section: str = None, 
                        endpoint: str = None, **kwargs):
        """Execute a test function with timing and error handling."""
        start_time = time.time()
        
        try:
            result = test_func(**kwargs)
            execution_time = time.time() - start_time
            
            if isinstance(result, dict) and result.get('responseStatus') == 'SUCCESS':
                self.log_test_result(test_name, 'SUCCESS', section, endpoint, 
                                   response_data=result, execution_time=execution_time)
            elif isinstance(result, dict) and result.get('responseStatus') == 'FAILURE':
                error_msg = result.get('errors', [{}])[0].get('message', 'API returned FAILURE')
                self.log_test_result(test_name, 'FAILURE', section, endpoint, 
                                   error=error_msg, response_data=result, execution_time=execution_time)
            else:
                self.log_test_result(test_name, 'SUCCESS', section, endpoint, 
                                   response_data=result, execution_time=execution_time)
            return result
            
        except Exception as e:
            execution_time = time.time() - start_time
            self.log_test_result(test_name, 'FAILURE', section, endpoint, 
                               error=str(e), execution_time=execution_time)
            return None
    
    def authenticate(self) -> bool:
        """Perform authentication and setup session."""
        auth_response = self.test_with_timing(
            test_func=self._do_authentication,
            test_name="Authentication",
            section="01",
            endpoint="/api/auth"
        )
        return auth_response is not None and auth_response.get('responseStatus') == 'SUCCESS'
    
    def _do_authentication(self):
        """Internal authentication implementation."""
        auth_response = self.auth_service.authenticate_with_username_password(
            username=VAULT_USERNAME,
            password=VAULT_PASSWORD,
            vaultDNS=VAULT_DNS
        )
        
        if auth_response.get('responseStatus') == 'SUCCESS':
            session_id = auth_response.get('sessionId')
            user_id = auth_response.get('userId')
            vault_id = auth_response.get('vaultId')
            
            # Update client with session info
            self.vault_client.sessionId = session_id
            self.vault_client.session_id = session_id
            self.vault_client.vaultId = vault_id
            self.vault_client.vaultDNS = VAULT_DNS
            
            self.session_active = True
        else:
            self.session_active = False
            
        return auth_response
    
    def print_summary(self):
        """Print test summary."""
        if not self.test_results:
            print("No tests executed yet.")
            return
            
        df = pd.DataFrame([{
            'test_name': r.test_name,
            'status': r.status,
            'section': r.section,
            'execution_time': r.execution_time
        } for r in self.test_results])
        
        status_counts = df['status'].value_counts()
        print("\n📊 TEST SUMMARY")
        print("=" * 40)
        for status, count in status_counts.items():
            emoji = {'SUCCESS': '✅', 'FAILURE': '❌', 'WARNING': '⚠️', 'SKIPPED': '⏭️'}.get(status, '❓')
            print(f"{emoji} {status}: {count}")
        
        if 'execution_time' in df.columns and df['execution_time'].notna().any():
            print(f"⏱️ Total time: {df['execution_time'].sum():.2f}s")
        print(f"🔐 Session: {'✅ Active' if self.session_active else '❌ Inactive'}")

print("✅ Baseline API testing framework initialized")

✅ Baseline API testing framework initialized


## Client Initialization and Authentication

Initialize the VaultClient and perform authentication to establish the testing session.

In [10]:
# Initialize the main vault client
print("🔧 Initializing VeevaVault client...")

vault_client = VaultClient()
vault_client.vaultURL = VAULT_URL
vault_client.vaultDNS = VAULT_DNS
vault_client.vaultUserName = VAULT_USERNAME
vault_client.vaultPassword = VAULT_PASSWORD

print(f"✅ VeevaVault client initialized")
print(f"🔗 Vault URL: {vault_client.vaultURL}")
print(f"📡 Vault DNS: {vault_client.vaultDNS}")
print(f"👤 Username: {vault_client.vaultUserName}")

# Initialize the baseline test framework
print("🔧 Initializing baseline test framework...")
baseline_tester = BaselineAPITester(vault_client)

print(f"✅ Baseline framework ready")

🔧 Initializing VeevaVault client...
✅ VeevaVault client initialized
🔗 Vault URL: https://vv-consulting-michael-mastermind.veevavault.com/
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔧 Initializing baseline test framework...
✅ Baseline framework ready


In [11]:
# Perform authentication 
print("🔐 Performing authentication...")

auth_success = baseline_tester.authenticate()

if auth_success:
    print(f"\n🎉 Authentication Successful!")
    print(f"🆔 Session ID: {vault_client.sessionId[:20]}..." if vault_client.sessionId else "No session ID")
    print(f"🏢 Vault ID: {vault_client.vaultId}")
    print("✅ Baseline template validated - ready for section-specific testing!")
else:
    print("❌ Authentication failed! Check credentials and try again.")

# Show summary
baseline_tester.print_summary()

🔐 Performing authentication...
✅ Authentication [01]: SUCCESS (0.33s)

🎉 Authentication Successful!
🆔 Session ID: 12413267BCB89E108E35...
🏢 Vault ID: 92425
✅ Baseline template validated - ready for section-specific testing!

📊 TEST SUMMARY
✅ SUCCESS: 1
⏱️ Total time: 0.33s
🔐 Session: ✅ Active


## Session Management and Next Steps

Basic session utilities and guidance for creating section-specific test notebooks.

### Available Functions:
- `get_session_info()` - Display current session status
- `end_session()` - Clean session logout
- `baseline_tester.print_summary()` - Show test results

### Create Section-Specific Notebooks:

1. **Copy this baseline template** 
2. **Import additional services** for the specific API section
3. **Add test methods** based on `api_docs/sections/XX_*.md`
4. **Follow test plans** in `notebook_tests/XX_*.md`
5. **Run comprehensive testing** for the section

**✅ Baseline template validated and ready for use!**

In [12]:
# Session management utilities
def get_session_info():
    """Get current session information."""
    print("🔍 SESSION INFO")
    print("="*30)
    print(f"Active: {'✅ Yes' if baseline_tester.session_active else '❌ No'}")
    print(f"Session ID: {vault_client.sessionId[:20] + '...' if vault_client.sessionId else 'None'}")
    print(f"Vault ID: {vault_client.vaultId or 'None'}")
    print(f"Tests Run: {len(baseline_tester.test_results)}")

def end_session():
    """End the current session."""
    if hasattr(baseline_tester.auth_service, 'logout') and baseline_tester.session_active:
        try:
            baseline_tester.auth_service.logout()
            baseline_tester.session_active = False
            vault_client.sessionId = None
            print("✅ Session ended")
        except Exception as e:
            print(f"❌ Logout failed: {str(e)}")
    else:
        print("⚠️ No active session or logout not available")

print("✅ Session utilities ready: get_session_info(), end_session()")

# Display current session
get_session_info()

✅ Session utilities ready: get_session_info(), end_session()
🔍 SESSION INFO
Active: ✅ Yes
Session ID: 12413267BCB89E108E35...
Vault ID: 92425
Tests Run: 1
